In [1]:
import pubchempy as pcp

# c_list = pcp.get_compounds("C6H12", "formula", record_type="3d")
search_name = "cyclopropane"
c_list = pcp.get_compounds(search_name, "name", record_type="3d")

c_list_clean = []
for c in c_list:
    if c.cid < 250000:
        c_list_clean.append(c)

for c in c_list_clean:
    print(c.cid, c.iupac_name, c.synonyms[0])
# c_list_clean.pop()
# for c in c_list_clean:
#     print(c.cid, c.iupac_name, c.synonyms[0])

6351 None CYCLOPROPANE


In [10]:
from pyscf.geomopt.geometric_solver import optimize
from pyscf import gto, scf, dft
import json
import copy
import json

with open("../cadft/utils/mol.json") as f:
    data = json.load(f)

dict_ = {}

for c in c_list_clean:
    name = search_name
    # if name is None:
    #     name = c.synonyms[0]
    # if name is None:
    #     print("No name for", c.cid)
    #     continue
    name = name.lower().replace(" ", "_")

    molecular = []
    for c_atom in c.atoms:
        atom = []
        atom.append(c_atom.element)
        atom.append(round(c_atom.x, 6))
        atom.append(round(c_atom.y, 6))
        atom.append(round(c_atom.z, 6))
        molecular.append(atom)

    mol = gto.M(atom=molecular, basis="CC-PVDZ")

    mdft = dft.RKS(mol)
    mdft.xc = "b3lyp"
    mdft.max_cycle = 110
    mdft.verbose = 4
    mdft.diis = None

    mol_eq = optimize(mdft, maxsteps=10, verbose=3)

    molecular_new = copy.deepcopy(molecular)
    for i, atom in enumerate(molecular_new):
        atom[1] = mol_eq.atom_coords(unit="ANG")[i][0]
        atom[2] = mol_eq.atom_coords(unit="ANG")[i][1]
        atom[3] = mol_eq.atom_coords(unit="ANG")[i][2]

    dict_[name] = molecular

data.update(dict_)

with open("../cadft/utils/mol.json", "w") as f:
    json.dump(data, f)

Create scanner for <class 'pyscf.grad.rks.Gradients'>
Create scanner for <class 'pyscf.dft.rks.RKS'>


geometric-optimize called with the following command line:
/home/dhem/anaconda3/envs/pyscf/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/dhem/.local/share/jupyter/runtime/kernel-v2-3498PMn9sunfG3LI.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              *****


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.318500   0.805700   0.000000    0.000000  0.000000  0.000000
   C  -0.857000  -0.127100   0.000000    0.000000  0.000000  0.000000
   C   0.538600  -0.678600   0.000000    0.000000  0.000000  0.000000
   H   0.532900   1.348300  -0.912000    0.000000  0.000000  0.000000
   H   0.532900   1.348300   0.912100    0.000000  0.000000  0.000000
   H  -1.434200  -0.212700   0.912000    0.000000  0.000000  0.000000
   H  -1.434200  -0.212600  -0.912100    0.000000  0.000000  0.000000
   H   0.901200  -1.135700  -0.912100    0.000000  0.000000  0.000000
   H   0.901200  -1.135600   0.912100    0.000000  0.000000  0.000000


******** <class 'pyscf.scf.hf.RKS_Scanner'> ********
method = RKS_Scanner
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS disabled
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 110
direct_scf = True
dir

RuntimeError: Nuclear gradients of <pyscf.scf.hf.RKS_Scanner object at 0x7ec710299580> not converged

In [3]:

import importlib.resources
from pathlib import Path
import os
import json

with importlib.resources.path("cadft", "utils") as resource_path:
    with open(
        Path(os.fspath(resource_path)) / "mol.json",
        "r",
        encoding="utf-8",
    ) as f:
        Mol = (json.load(f))
print(Mol.keys())

/tmp/ipykernel_131263/2901700228.py:6: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with importlib.resources.path("cadft", "utils") as resource_path:


MAIN_PATH: /home/dhem/workspace/2024.1
DATA_PATH: /home/dhem/workspace/2024.1/data/grids_mrks
DATA_SAVE_PATH: /home/dhem/workspace/2024.1/data/grids_mrks/saved_data
DATA_CC_PATH: /home/dhem/workspace/2024.1/data/test
dict_keys(['hexane', 'Cyclopentane', '2-methylpentane', '3-methylpentane', '2,2-dimethylbutane', '2,3-dimethylbutane', 'cyclohexane', '1-hexene', 'methylcyclopentane', '3,3-dimethyl-1-butene', '4-methyl-1-pentene', '2-methyl-1-pentene', '2-hexene', '2,3-dimethyl-2-butene', '3-hexene', '2-ethyl-1-butene', 'propylcyclopropane', '2,3-dimethyl-1-butene', '2-methyl-2-pentene', '4-methyl-2-pentene', '3-methyl-1-pentene', '2-pentene, 3-methyl-', '1-ethyl-1-methylcyclopropane', 'methane', 'ethane', 'ethylene', 'acetylene', 'cyclopentane', 'neopentane', 'isopentane', 'pentane', 'benzene', 'spiropentane', 'cyclobutane', 'cyclopropylmethyl', 'butyne', 'butadiene', 'butane', 'isobutane', 'propane', 'propyne', 'cyclopropene', 'propylene'])
